# 4. Errors and Exception Handling

An exception is an error that happens during the execution of a program. Exceptions are known to non-programmers as instances that do not conform to a general rule. The name "exception" in computer science has this meaning as well: It implies that the problem (the exception) doesn't occur frequently, i.e. the exception is the "exception to the rule". **Exception handling is a construct in some programming languages to handle or deal with errors automatically**.

Error handling is generally resolved by saving the state of execution at the moment the error occurred and interrupting the normal flow of the program to execute a special function or piece of code, which is known as the exception handler. Depending on the kind of error ("division by zero", "file open error" and so on) which had occurred, the error handler can "fix" the problem and the programm can be continued afterwards with the previously saved data.

## 4.1 Syntax

The syntax of python error handling contains four key word [try, except, else, finally]. Below figure shows the specification of each block

![py_try_except.PNG](../../../images/py_try_except.PNG)



In [1]:
n = int(input("Please enter a number: "))

ValueError: invalid literal for int() with base 10: 'toto'

## 4.2 Simple example

With the aid of exception handling, we can write robust code for reading an integer from input:

In [3]:
n=None
while True:
    try:
        n=int(input("Please enter a number: "))
        break
    except ValueError as e:
        print(f"You must enter a number. Please try again origin message:{e}")
print(f"Great you have entered {n}")

You must enter a number. Please try again origin message:invalid literal for int() with base 10: 'dff'
You must enter a number. Please try again origin message:invalid literal for int() with base 10: 'df'
Great you have entered 18


We can write a function to reuse the above code

In [4]:
def get_number(prompt_msg):
    while True:
        try:
            n=int(input(prompt_msg))
            # we don't need to call break, return will stop the loop
            return n
        except ValueError as e:
            print(f"You must enter a number. Please try again origin message:{e}")

In [5]:
# this function convert a dog age to a equivalent humain age
def dog2human_age(dog_age):
    human_age = -1
    if dog_age < 0:
        human_age = -1
    elif dog_age == 0:
        human_age = 0
    elif dog_age == 1:
        human_age = 14
    elif dog_age == 2:
        human_age = 22
    else:
        human_age = 22 + (dog_age -2) * 5
    return human_age

In [6]:
dog_age=get_number("Please enter your dog age?")
print(f"your dog in humain age: {dog2human_age(dog_age)}")

your dog in humain age: 87


## 4.3 Multiple Except clause

A try statement may have more than one except clause for different exceptions. But at most one except clause will be executed.

Our next example shows a try clause, in which we open a file for reading, read a line from this file and convert this line into an integer. There are at least two possible exceptions:

In [9]:
import sys
path="../../../sources/numbers1.txt"
try:
    f=open(path,"r")
    text=f.read()
    number=int(text)
    print(f"Read number {number}")
except IOError as e:
    errno, strerror = e.args
    print(f"I/O error({errno}): {strerror}")
except ValueError:
    print("Value Error")
except:
    print("Unexpected error:", sys.exc_info()[0])
    raise

I/O error(2): No such file or directory


In above example, the variable "e" is bound to an exception instance with the arguments stored in instance.args. If we call the above script with a non-existing file, we get the message: I/O error(2): No such file or directory

And if the file integers.txt is not readable, e.g. if we don't have the permission to read it, we get the following message: I/O error(13): Permission denied

An except clause may name more than one exception in a tuple of error names, as we see in the following example:

In [ ]:
try:
    f=open(path)
    text=f.read()
    number=int(text)
except (IOError, ValueError):
    print("An I/O error or a ValueError occurred")
except:
    print("An unexpected error occurred")
    raise


**Order matters when we have multiple except clause.** In section 4.5. We will see the hierarchy of exceptions. If the upper except catch a parent class exception, and lower except catches a child class exception. The lower except clause will never be executed. Consider below example:
```python
try:
    foo()
except ValueError as e:
    print('ValueError')
except UnicodeError as e:
    print('UnicodeError')
```

The second except is useless, because UnicodeError exception is a sub class of ValueError. So the first except will also catch the UnicodeError.

## 4.4 Exception propagation

Let's try below example, the exception is thrown by a statement inside a function. We called the function inside another try. It means **if we don't handle the exception, the exception will be propagated to an upper leve automatically. If the exception has been propagated to the highest level, and still not handled. The python interpreter will catch the exception and print error traceback stack.** That's a life-saving feature, because we don't need to handle the exception at each level unlike Java(we need to throw it at each level if we don't to handle it)

In [12]:
def convert():
    return int("four")

try:
    convert()
    print("convert success")
except ValueError:
    print("catch in main")

catch in main


Now let's try to handle the exception inside the function. You can notice if an exception is handled, no more propagation. And the function is considered success.

In [13]:
def convert():
    try:
        int("four")
    except ValueError:
        print("catch in function")

try:
    convert()
    print("convert success")
except ValueError:
    print("catch in main")

catch in function
convert success


If you want to propagate the exception even after it has been handled (Not recommended), You can use key word **raise**. Check below example, you can notice, we catch two exception, one local, one in main. And as the convert() throws exception, it's no longer considered as success.

In [14]:
def convert():
    try:
        int("four")
    except ValueError:
        print("catch in function")
        # propagate the caught exception
        raise

try:
    convert()
    print("convert success")
except ValueError:
    print("catch in main")

catch in function
catch in main


## 4.5 Custom exception

It's possible to create Exceptions yourself if the built-in exception does not fit your requirements. Because all exceptions in python are classes or sub Classes of BaseException. You can find the full list and their dependencies [here](https://docs.python.org/3/library/exceptions.html#exception-hierarchy)

All your need to do is just extends another exception(e.g. Exception)

Check below example, we create a MyException class which extends the Exception class. Even though it's empty, it inherits all methods of its partent.

In [15]:
class MyException(Exception):
    pass

In [16]:
def throw_exception(flag):
    if flag:
        raise MyException("This is a custom define Exception thrown in a function")

In [17]:
throw_exception(True)

MyException: This is a custom define Exception thrown in a function

## 4.6 Clean-up Actions (try ... finally)

So far the try statement had always been paired with except clauses. But there is another way to use it as well. The try statement can be followed by a finally clause. Finally clauses are called clean-up or termination clauses, because they must be executed under all circumstances, i.e. a "finally" clause is always executed regardless if an exception occurred in a try block or not. A simple example to demonstrate the finally clause:

If you enter a number, the finally statement is executed. If you give a word, the finally statement is executed too.

In [18]:
try:
    x = float(input("Your number: "))
    inverse = 1.0 / x
finally:
    print("There may or may not have been an exception.")
print("The inverse: ", inverse)

There may or may not have been an exception.


ValueError: could not convert string to float: 'toto'

"finally" and "except" can be used together for the same try block, as it can be seen in the following Python example:

In [20]:
try:
    num=int(input("Enter a number"))
    print(f"input : {num}")
    reversed= 1.0/num
except ValueError:
    print("Must enter a number, please try again")
except ZeroDivisionError:
    print("Infinite")
finally:
    print("There may or may not have been an exception.")

input : 0
Infinite
There may or may not have been an exception.


# 4.7 Else

The try ... except statement has an optional else clause. **An else block has to be positioned after all the except clauses. An else clause will be executed if the try clause doesn't raise an exception.**

The following example opens a file and reads in all the lines into a list called "text". We handled the exception, but there are two problems:
 1. readlines and close can also throw exception.
 2. if an exception occurred, the f.close() will never be executed, and the file connexion is still open. If we have many unclosed connexion, this will kill our performance.



In [ ]:
path="../../../sources/numbers.txt"
try:
    f=open(path,"r")
    text=f.readlines()
    print(f"file content: {text}")
    f.close()
except IOError as e:
    print(e)

With the help of else and finally clause, we can use below code to resolve the two problems

In [21]:
path="../../../sources/numbers.txt"
f=None
try:
    f=open(path,"r")
except IOError as e:
    print(e)
else:
    text=f.readlines()
    print(f"file content: {text}")
finally:
    f.close()

file content: ['12']


## 4.8 Understand the python error Traceback stack

Try to run the `src/err.py`. You should see below error message in your python console

```text
Traceback (most recent call last):
  File "/home/pliu/git/Learning_Python/learning_python/Lesson01_Basics/Section01_Basic_syntax/src/err.py", line 14, in <module>
    main()
  File "/home/pliu/git/Learning_Python/learning_python/Lesson01_Basics/Section01_Basic_syntax/src/err.py", line 10, in main
    bar('0')
  File "/home/pliu/git/Learning_Python/learning_python/Lesson01_Basics/Section01_Basic_syntax/src/err.py", line 6, in bar
    return foo(s) * 2
  File "/home/pliu/git/Learning_Python/learning_python/Lesson01_Basics/Section01_Basic_syntax/src/err.py", line 2, in foo
    return 10 / int(s)
ZeroDivisionError: division by zero
```


Let's explain it line by line

### Line 1：

```text
Traceback (most recent call last):
```

This tells us below are traceback info about the error

### line 2~3：

```text
File "/home/pliu/git/Learning_Python/learning_python/Lesson01_Basics/Section01_Basic_syntax/src/err.py", line 14, in <module>
    main()
```

The error occurs when we call main() function, which located at line 14 in file `err.py`

### line 4~5:

```text
File "/home/pliu/git/Learning_Python/learning_python/Lesson01_Basics/Section01_Basic_syntax/src/err.py", line 10, in main
    bar('0')
```
It tells you the error in main is caused by the call of **bar('0')** in line 10


### line 6~7:

```text
File "/home/pliu/git/Learning_Python/learning_python/Lesson01_Basics/Section01_Basic_syntax/src/err.py", line 6, in bar
    return foo(s) * 2
```

It tells you the error in **bar('0')** is caused by the call of **foo(s)\*2** in line 6

### line 8~10

```text
File "/home/pliu/git/Learning_Python/learning_python/Lesson01_Basics/Section01_Basic_syntax/src/err.py", line 2, in foo
    return 10 / int(s)
ZeroDivisionError: division by zero
```

It tells you the error in **foo(s)** is caused by the call of **return 10 / int(s)** in line 2. And the origin exception is **ZeroDivisionError: division by zero**

### Conclusion

The type conversion is correct, but we can't divide 10 by 0.

## 4.9 Saving the error message

We have seen how to handle the error, but we can imagine in a real world app, we can't stop the app and check what's going on. We need to save the error message and keep the app running.

So we need a logging system, which we will talk about it in next chapter